In [ ]:
lulc_rst = '/mnt/e/autocls01/dadosfrancisco/fidx/lulc_ref.tif'

idx_rst = {
    "20220504" : {
        "ndvi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndvi_20220504.tif',
        "ndbi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndbi_20220504.tif',
        "ndwi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndwi_20220504.tif'
    },
    "20220812" : {
        "ndvi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndvi_20220812.tif',
        "ndbi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndbi_20220812.tif',
        "ndwi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndwi_20220812.tif'
    },
    "20221001" : {
        "ndvi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndvi_20221001.tif',
        "ndbi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndbi_20221001.tif',
        "ndwi" : '/mnt/e/autocls01/dadosfrancisco/fidx/ndwi_20221001.tif'
    }
}

idx_rules = {
    1 : {
        "ndbi" : {'operator' : '>', 'value': 0, 'all' : None},
        "ndvi" : {'operator' : '<', 'value': 0.3, 'all': True},
        "ndwi" : {'operator' : '<', 'value': 0, 'all' : True}
    },
    2 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3, 'all': True},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': True}
    },
    3 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3, 'all': True},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': True}
    },
    4 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3, 'all': True},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': True}
    },
    5 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3, 'all': True},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': True}
    },
    6 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0, 'all': None},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': None}
    },
    7 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0, 'all': None},
        "ndwi" : {'operator' : '<', 'value': 0, 'all': None}
    },
    8 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '<', 'value': 0.3, 'all': None},
        "ndwi" : {'operator' : '>', 'value': 0, 'all': True}
    }
}

fraster= '/mnt/e/autocls01/dadosfrancisco/fidx_res/lulc_filter.tif'

In [ ]:
import os
from osgeo import gdal
import numpy as np
from glass.rd.rst import rst_to_array
from glass.prop.rst import get_nodata
from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str

In [ ]:
# Check if outfolder exists
out_folder = os.path.dirname(fraster)
if not os.path.exists(out_folder):
    mkdir(out_folder, overwrite=None)
    
"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    out_folder, grassBIN='grass78', location=loc,
    srs=lulc_rst
    )

import grass.script.setup as gsetup
    
gsetup.init(grsb, out_folder, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from grass.pygrass.modules import Module

In [ ]:
# Identificar classes existentes no raster com as classes de LULC

lulc_img = rst_to_array(lulc_rst)

ndval = get_nodata(lulc_rst)

lulc_class = [v for v in np.unique(lulc_img) if v != ndval]

In [ ]:
# Send all rasters to GRASS GIS

lulc_grs = rst_to_grs(lulc_rst, fprop(lulc_rst, 'fn'))

for d in idx_rst:
    for k in idx_rst[d]:
        idx_rst[d][k] = rst_to_grs(idx_rst[d][k], fprop(idx_rst[d][k], 'fn'))

Para cada classe, índice e data, criar um ficheiro que indica se o pixel deve ser considerado para treino (1) ou não (0)

In [ ]:
mask_by_class_day_idx = {}

for cls in lulc_class:
    mask_by_class_day_idx[cls] = {}
    for d in idx_rst:
        mask_by_class_day_idx[cls][d] = {}
        for k in idx_rst[d]:
            if not idx_rules[cls][k]: continue
            
            form = (
                f"if({lulc_grs} == {str(cls)} && "
                f"{idx_rst[d][k]} {idx_rules[cls][k]['operator']} "
                f"{idx_rules[cls][k]['value']}, 1, 0)"
            )
            
            outmask = f'mask_{cls}_{k}_{d}'
            rc = Module ('r.mapcalc', expression=f'{outmask} = {form}')
            
            mask_by_class_day_idx[cls][d][k] = outmask
            
            grs_to_rst(outmask, os.path.join(
                os.path.dirname(fraster),
                f'{outmask}.tif'
            ), rtype=int)

Para cada classe e índice, criar um ficheiro que indica se o pixel deve ser incluído no treino (1) ou não (0), tendo em conta as várias máscaras obtidas para essa classe e índice em cada uma das datas consideradas

In [ ]:
cls_masks = {}

for cls in mask_by_class_day_idx:
    cls_masks[cls] = {}
    
    for d in mask_by_class_day_idx[cls]:
        for idx in mask_by_class_day_idx[cls][d]:
            if idx not in cls_masks[cls]:
                cls_masks[cls][idx] = []
            
            cls_masks[cls][idx].append(mask_by_class_day_idx[cls][d][idx])

In [ ]:
cls_masks

In [ ]:
for cls in cls_masks:
    for idx in cls_masks[cls]:
        ifr = [f"{r} == 1" for r in cls_masks[cls][idx]]
        if idx_rules[cls][idx]['all']:
            
            _if = " && ".join(ifr)
        
        else:
            _if = " || ".join(ifr)
        
        exp = f"if({_if}, 1, 0)"
        
        omask = f'clsidxmask_{cls}_{idx}'
        rc = Module('r.mapcalc', expression=f'{omask} = {exp}')
        
        cls_masks[cls][idx] = omask

In [ ]:
cls_masks

Obter máscara binária final para cada classe

In [ ]:
for cls in cls_masks:
    ifr = [f"{cls_masks[cls][idx]} == 1" for idx in cls_masks[cls]]
    
    exp = f"if({' && '.join(ifr)}, 1, 0)"
    
    om = f"clsmask_{cls}"
    rc = Module('r.mapcalc', expression=f'{om} = {exp}')
    
    cls_masks[cls] = om

In [ ]:
cls_masks

Obter ficheiro que indica as classes a considerar no treino - neste raster, o valor 0 indica que o pixel não deve ser considerado no treino.

In [ ]:
ifs = []

for c in cls_masks:
    exp = (
        f"if({cls_masks[c]} == 1, "
        f"{str(c)}, "
        f"{'0' if not len(ifs) else ifs[len(ifs)-1]})"
    )
    
    ifs.append(exp)

In [ ]:
ifs[-1]